In [4]:
import numpy as np
import tensorflow as tf

In [6]:
class NNmodel2():
    '''[summary]
    
    [description]
    '''

    def __init__(self, inpSize, opSize, layers, activations):
        

        self.modelOK = False
        self.checkPoint = None
        self.optimizer = None

        self.fitted = False
        self.currentErrors = None


        self.init    = tf.global_variables_initializer()
        self.inpSize = inpSize
        self.Inp     = tf.placeholder(dtype=tf.float32, shape=inpSize, name='Inp')
        self.Op      = tf.placeholder(dtype=tf.float32, shape=opSize, name='Op')

        self.allW         = []
        self.allWPH       = []
        self.allAssignW   = []

        self.allB         = []
        self.allBPH       = []
        self.allAssignB   = []


        self.result  = None

        prevSize = inpSize[0]
        for i, l in enumerate(layers):

            tempW       = tf.Variable( 0.1*(np.random.rand(l, prevSize) - 0.5), dtype=tf.float32, name='W_{}'.format(i) )
            tempWPH     = tf.placeholder(dtype=tf.float32, shape=(l, prevSize), name='PHW_{}'.format(i))
            tempAssignW = tf.assign(tempW, tempWPH, name='AssignW_{}'.format(i))

            tempB       = tf.Variable( 0, dtype=tf.float32, name='B_{}'.format(i) )
            tempBPH     = tf.placeholder(dtype=tf.float32, shape=tuple(), name='PHB_{}'.format(i))
            tempAssignB = tf.assign(tempB, tempBPH, name='AssignB_{}'.format(i))

            self.allW.append( tempW )
            self.allWPH.append( tempWPH )
            self.allAssignW.append( tempAssignW )

            self.allB.append( tempB )
            self.allBPH.append( tempBPH )
            self.allAssignB.append( tempAssignB )


            if i == 0:
                self.result = tf.matmul( tempW, self.Inp ) + tempB
            else:
                self.result = tf.matmul( tempW, self.result ) + tempB

            prevSize = l

            if activations[i] is not None:
                self.result = activations[i]( self.result )

        self.err = tf.sqrt(tf.reduce_mean((self.Op - self.result)**2))
        self.modelOK = True



        return

In [7]:
X = np.random.rand(2, 10000)
y = (  2*np.sin(X[0, :]) + 3*np.cos(X[1, :]) ).reshape(1, -1)

print('We are in the NNmodule')
inpSize     = (2, None)
opSize      = (1, None)
layers      = (5, 8, 1)
activations = [tf.tanh, tf.tanh, None]
model1      = NNmodel2(inpSize, opSize, layers, activations)


We are in the NNmodule


In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [10]:
weights = sess.run(model1.allW + model1.allB)

In [15]:
[ w.shape for w in weights]

[(5, 2), (8, 5), (1, 8), (), (), ()]

In [16]:
weights1 = [w+3 for w in weights]
print(weights1)

[array([[ 3.03136492,  3.00360036],
       [ 3.03496718,  3.03835034],
       [ 3.00998712,  3.02417254],
       [ 3.01572347,  3.03830409],
       [ 2.98062611,  3.03701091]], dtype=float32), array([[ 2.97221661,  3.0409236 ,  2.99432778,  2.98048902,  3.00026703],
       [ 3.01601267,  3.04845738,  2.97617173,  2.95996666,  2.96442676],
       [ 2.96671319,  2.98540378,  3.03539729,  2.95815921,  2.95071554],
       [ 2.98195243,  2.97560239,  2.99858451,  3.02192783,  3.00379157],
       [ 2.98031282,  3.01286554,  3.01413441,  2.95104742,  3.03247023],
       [ 2.99555945,  2.99656153,  2.9828701 ,  2.99992323,  2.954638  ],
       [ 3.02651358,  2.97189689,  3.02862167,  2.98879504,  2.99394393],
       [ 3.04076362,  2.98958564,  3.0087049 ,  2.96900654,  2.97659087]], dtype=float32), array([[ 2.96111417,  3.00943494,  2.97367597,  3.01174617,  2.99469614,
         2.99240255,  2.98095012,  2.98283744]], dtype=float32), 3.0, 3.0, 3.0]


In [17]:
Ws = sess.run(model1.allW)
Ws1 = [w+5 for w in Ws]

In [22]:
for i, (w, mW) in enumerate(zip(Ws1, model1.allAssignW)):
    sess.run(mW, feed_dict={ 'PHW_{}:0'.format(i) : w } )

In [23]:
sess.run(model1.allW)

[array([[ 5.03136492,  5.0036006 ],
        [ 5.03496742,  5.03835058],
        [ 5.00998735,  5.02417231],
        [ 5.01572371,  5.03830433],
        [ 4.98062611,  5.03701067]], dtype=float32),
 array([[ 4.97221661,  5.0409236 ,  4.99432755,  4.98048878,  5.00026703],
        [ 5.01601267,  5.04845762,  4.97617149,  4.95996666,  4.96442699],
        [ 4.96671343,  4.98540354,  5.03539753,  4.95815897,  4.95071554],
        [ 4.98195219,  4.97560215,  4.99858427,  5.02192783,  5.00379181],
        [ 4.98031282,  5.01286554,  5.01413441,  4.95104742,  5.03247023],
        [ 4.99555969,  4.99656153,  4.9828701 ,  4.99992323,  4.954638  ],
        [ 5.02651358,  4.97189713,  5.02862167,  4.98879528,  4.99394417],
        [ 5.04076338,  4.98958588,  5.00870514,  4.96900654,  4.97659063]], dtype=float32),
 array([[ 4.96111441,  5.00943518,  4.97367573,  5.01174593,  4.99469614,
          4.99240255,  4.98094988,  4.9828372 ]], dtype=float32)]

In [24]:
sess.close()

In [25]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i, (w, mW) in enumerate(zip(Ws1, model1.allAssignW)):
    sess.run(mW, feed_dict={ 'PHW_{}:0'.format(i) : w } )

print(sess.run(model1.allW))

sess.close()

[array([[ 5.03136492,  5.0036006 ],
       [ 5.03496742,  5.03835058],
       [ 5.00998735,  5.02417231],
       [ 5.01572371,  5.03830433],
       [ 4.98062611,  5.03701067]], dtype=float32), array([[ 4.97221661,  5.0409236 ,  4.99432755,  4.98048878,  5.00026703],
       [ 5.01601267,  5.04845762,  4.97617149,  4.95996666,  4.96442699],
       [ 4.96671343,  4.98540354,  5.03539753,  4.95815897,  4.95071554],
       [ 4.98195219,  4.97560215,  4.99858427,  5.02192783,  5.00379181],
       [ 4.98031282,  5.01286554,  5.01413441,  4.95104742,  5.03247023],
       [ 4.99555969,  4.99656153,  4.9828701 ,  4.99992323,  4.954638  ],
       [ 5.02651358,  4.97189713,  5.02862167,  4.98879528,  4.99394417],
       [ 5.04076338,  4.98958588,  5.00870514,  4.96900654,  4.97659063]], dtype=float32), array([[ 4.96111441,  5.00943518,  4.97367573,  5.01174593,  4.99469614,
         4.99240255,  4.98094988,  4.9828372 ]], dtype=float32)]
